In [1]:
from MFEA_lib.tasks.Benchmark.IDPC_EDU import IDPC_EDU_benchmark
from MFEA_lib.model import MFEA_base
from MFEA_lib.model.utils import *
from MFEA_lib.operators.Crossover import *
from MFEA_lib.operators.Mutation import *
from MFEA_lib.operators.Selection import *

In [2]:
tasks, IndClass = IDPC_EDU_benchmark.get_tasks(1)

  0%|          | 0/24 [00:00<?, ?it/s]

Reading data...


100%|██████████| 24/24 [00:07<00:00,  3.11it/s]


In [3]:
tasks

In [4]:
baseModel = MFEA_base.model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover= newSBX(nc = 2, gamma= 0.4, alpha= 6),
    crossover= NoCrossover(nc = 2),
    mutation= NoMutation(nm = 5),
    selection= ElitismSelection()
)
solve = baseModel.fit(
    nb_generations = 1000, rmp = 0.3, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= False
)

Time: 00m 22.48s   13 % [==>                 ]  Cost: 4.60E+01  2.10E+01  4.20E+01  4.90E+01  2.60E+01  1.20E+01  3.40E+01  1.80E+01  3.20E+01  3.80E+01  4.60E+01  3.90E+01  3.60E+01  2.20E+01  1.30E+01  3.30E+01  4.20E+01  1.60E+01  1.60E+01  4.60E+01  1.20E+01  2.60E+01  2.10E+01  2.80E+01  ,  

KeyboardInterrupt: 